# Import Libraries

In [27]:
!jupyter nbextension install --user --py pythreejs
!jupyter nbextension enable --user --py pythreejs

Installing /mnt/shared/ms/anaconda3/envs/py36/lib/python3.6/site-packages/pythreejs/static -> jupyter-threejs
Up to date: /mnt/shared/ms/.local/share/jupyter/nbextensions/jupyter-threejs/index.js
Up to date: /mnt/shared/ms/.local/share/jupyter/nbextensions/jupyter-threejs/index.js.map
Up to date: /mnt/shared/ms/.local/share/jupyter/nbextensions/jupyter-threejs/extension.js
Up to date: /mnt/shared/ms/.local/share/jupyter/nbextensions/jupyter-threejs/three.js
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable pythreejs --user --py
    
Enabling notebook extension jupyter-threejs/extension...
      - Validating: OK


In [28]:
import os
import numpy as np
from lib.datasets.kitti_dataset import KittiDataset
from lib.datasets.kitti_rcnn_dataset import KittiRCNNDataset
from lib.utils import kitti_utils
import argoverse
from argoverse.data_loading.argoverse_tracking_loader import ArgoverseTrackingLoader
from scipy.spatial.transform import Rotation
path = 'output/rpn/argo_config/eval/epoch_120/val/seg_result/'

In [29]:
from pyntcloud import PyntCloud
from pyntcloud.io import bin as io_bin
import numpy as np
import pandas as pd
import os

from pythreejs import *
import numpy as np
from IPython.display import display
from ipywidgets import HTML, Text, Output, VBox
from traitlets import link, dlink
from glob import glob

## To Visualize the Segmentation Results(RPN)

In [30]:
seg_result = os.listdir(path)
seg_idx_list = [int(f.split('.')[0]) for f in seg_result]
seg_idx_list.sort()

In [31]:
def get_seg_lidar(idx):
    return np.load(os.path.join(path,str(idx).zfill(6) + '.npy'))


In [32]:
def visualize(pc,bbox = []):
    if len(bbox) == 0:
        new_pts = pd.DataFrame(pc.astype('float32'), columns=['x', 'y', 'z'])
        cloud = PyntCloud(new_pts)
        cloud.plot(initial_point_size=0.2)
    else:
        new_pts = pd.DataFrame(pc.astype('float32'), columns=['x', 'y', 'z'])
        cloud = PyntCloud(new_pts)
        cloud.plot(initial_point_size=0.2, polylines = bbox)

In [33]:
def createBBox(bounding_box,C1,C2,C3,C4,C5,C6,C7,C8,color="yellow"):
    bounding_box.append(
        {
            "color":color,
            "vertices":[C1,C2,C3,C4,C1]
        })
    bounding_box.append(
        {
            "color":color,
            "vertices":[C1,C4,C8,C5,C1]
        })
    bounding_box.append(
        {
            "color":color,
            "vertices":[C1,C2,C6,C5,C1]
        })
    bounding_box.append(
        {
            "color":color,
            "vertices":[C2,C6,C7,C3,C2]
        })
    bounding_box.append(
        {
            "color":color,
            "vertices":[C3,C7,C8,C4,C3]
        })
    return bounding_box

In [34]:
root_dir = '/data/Argoverse/argoverse-tracking'

In [35]:
dataloader = KittiRCNNDataset(root_dir, split = 'val')
#foreground_mask = (sample['rpn_cls_label'] == 1)
#background_mask = (sample['rpn_cls_label'] == 0)
#fg = sample['pts_input'][foreground_mask]
#bg = sample['pts_input'][background_mask]

In [36]:
#gt_boxes3d = sample['gt_boxes3d']
#gt_corners = kitti_utils.boxes3d_to_corners3d(gt_boxes3d, rotate=True)

# Visualize Final Bboxes

In [38]:
# Set the value of pc idx to visualize that particular idx of validation data
pc_idx = 10
orig_pc = dataloader.get_lidar(pc_idx)
sample = dataloader.get_rpn_sample(pc_idx)
dataloader.lidar_pathlist

['/data/Argoverse/argoverse-tracking/val/22160544_2216_2216_2216_722161741824/lidar/PC_315966714120350000.ply',
 '/data/Argoverse/argoverse-tracking/val/22160544_2216_2216_2216_722161741824/lidar/PC_315966714219883000.ply',
 '/data/Argoverse/argoverse-tracking/val/22160544_2216_2216_2216_722161741824/lidar/PC_315966714320079000.ply',
 '/data/Argoverse/argoverse-tracking/val/22160544_2216_2216_2216_722161741824/lidar/PC_315966714420276000.ply',
 '/data/Argoverse/argoverse-tracking/val/22160544_2216_2216_2216_722161741824/lidar/PC_315966714520472000.ply',
 '/data/Argoverse/argoverse-tracking/val/22160544_2216_2216_2216_722161741824/lidar/PC_315966714620005000.ply',
 '/data/Argoverse/argoverse-tracking/val/22160544_2216_2216_2216_722161741824/lidar/PC_315966714720201000.ply',
 '/data/Argoverse/argoverse-tracking/val/22160544_2216_2216_2216_722161741824/lidar/PC_315966714820398000.ply',
 '/data/Argoverse/argoverse-tracking/val/22160544_2216_2216_2216_722161741824/lidar/PC_31596671491993100

In [39]:
rcnn_path = './output/rcnn/argo_config_sampling/eval/epoch_no_number/val/final_result/data/'

In [40]:
def cls_type_to_id(cls_type):
    type_to_id = {'VEHICLE': 1, 'Pedestrian': 2, 'Cyclist': 3, 'Van': 4}
    if cls_type not in type_to_id.keys():
        return -1
    return type_to_id[cls_type]


class Object3d_kitti(object):
    def __init__(self, line):
        label = line.strip().split(' ')
        self.src = line
        self.cls_type = label[0]
        self.cls_id = cls_type_to_id(self.cls_type)
        self.trucation = float(label[1])
        self.occlusion = float(label[2])  # 0:fully visible 1:partly occluded 2:largely occluded 3:unknown
        self.alpha = float(label[3])
        self.box2d = np.array((float(label[4]), float(label[5]), float(label[6]), float(label[7])), dtype=np.float32)
        self.h = float(label[8])
        self.w = float(label[9])
        self.l = float(label[10])
        self.pos = np.array((float(label[11]), float(label[12]), float(label[13])), dtype=np.float32)
        self.dis_to_cam = np.linalg.norm(self.pos)
        self.ry = float(label[14])
        self.score = float(label[15]) if label.__len__() == 16 else -1.0
        self.level_str = None
        self.level = self.get_obj_level()

    def get_obj_level(self):
        height = float(self.box2d[3]) - float(self.box2d[1]) + 1

        if height >= 40 and self.trucation <= 0.15 and self.occlusion <= 0:
            self.level_str = 'Easy'
            return 1  # Easy
        elif height >= 25 and self.trucation <= 0.3 and self.occlusion <= 1:
            self.level_str = 'Moderate'
            return 2  # Moderate
        elif height >= 25 and self.trucation <= 0.5 and self.occlusion <= 2:
            self.level_str = 'Hard'
            return 3  # Hard
        else:
            self.level_str = 'UnKnown'
            return 4

    def generate_corners3d(self):
        """
        generate corners3d representation for this object
        :return corners_3d: (8, 3) corners of box3d in camera coord
        """
        l, h, w = self.l, self.h, self.w
        x_corners = [l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2]
        y_corners = [h/2, h/2, h/2, h/2, -h/2, -h/2, -h/2, -h/2]
        z_corners = [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2]

        R = np.array([[np.cos(self.ry), 0, np.sin(self.ry)],
                      [0, 1, 0],
                      [-np.sin(self.ry), 0, np.cos(self.ry)]])
        corners3d = np.vstack([x_corners, y_corners, z_corners])  # (3, 8)
        corners3d = np.dot(R, corners3d).T
        corners3d = corners3d + self.pos
        return corners3d

    def to_bev_box2d(self, oblique=True, voxel_size=0.1):
        """
        :param bev_shape: (2) for bev shape (h, w), => (y_max, x_max) in image
        :param voxel_size: float, 0.1m
        :param oblique:
        :return: box2d (4, 2)/ (4) in image coordinate
        """
        if oblique:
            corners3d = self.generate_corners3d()
            xz_corners = corners3d[0:4, [0, 2]]
            box2d = np.zeros((4, 2), dtype=np.int32)
            box2d[:, 0] = ((xz_corners[:, 0] - Object3d.MIN_XZ[0]) / voxel_size).astype(np.int32)
            box2d[:, 1] = Object3d.BEV_SHAPE[0] - 1 - ((xz_corners[:, 1] - Object3d.MIN_XZ[1]) / voxel_size).astype(np.int32)
            box2d[:, 0] = np.clip(box2d[:, 0], 0, Object3d.BEV_SHAPE[1])
            box2d[:, 1] = np.clip(box2d[:, 1], 0, Object3d.BEV_SHAPE[0])
        else:
            box2d = np.zeros(4, dtype=np.int32)
            # discrete_center = np.floor((self.pos / voxel_size)).astype(np.int32)
            cu = np.floor((self.pos[0] - Object3d.MIN_XZ[0]) / voxel_size).astype(np.int32)
            cv = Object3d.BEV_SHAPE[0] - 1 - ((self.pos[2] - Object3d.MIN_XZ[1]) / voxel_size).astype(np.int32)
            half_l, half_w = int(self.l / voxel_size / 2), int(self.w / voxel_size / 2)
            box2d[0], box2d[1] = cu - half_l, cv - half_w
            box2d[2], box2d[3] = cu + half_l, cv + half_w

        return box2d

    def to_str(self):
        print_str = '%s %.3f %.3f %.3f box2d: %s hwl: [%.3f %.3f %.3f] pos: %s ry: %.3f' \
                     % (self.cls_type, self.trucation, self.occlusion, self.alpha, self.box2d, self.h, self.w, self.l,
                        self.pos, self.ry)
        return print_str

    def to_kitti_format(self):
        kitti_str = '%s %.2f %d %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f' \
                    % (self.cls_type, self.trucation, int(self.occlusion), self.alpha, self.box2d[0], self.box2d[1],
                       self.box2d[2], self.box2d[3], self.h, self.w, self.l, self.pos[0], self.pos[1], self.pos[2],
                       self.ry)
        return kitti_str

class Object3d(object):
    def __init__(self, line):
        self.argo_to_kitti = np.array([[6.927964e-03, -9.999722e-01, -2.757829e-03],
                                       [-1.162982e-03, 2.749836e-03, -9.999955e-01],
                                       [9.999753e-01, 6.931141e-03, -1.143899e-03]])
        
        label = line
        self.src = line
        self.cls_type = label['label_class']
        self.cls_id = cls_type_to_id(self.cls_type)
        
        self.trucation = 0.0
        self.occlusion = 0.0  
        self.alpha = np.arctan2(label['center']['z'],label['center']['x'])
        
        self.h = float(label['height'])
        self.w = float(label['width'])
        self.l = float(label['length'])
        self.pos_argo = np.array([float(label['center']['x']), float(label['center']['y']), float(label['center']['z'])], dtype=np.float32)
        
        #KITTI Frame
        self.pos = np.dot(self.argo_to_kitti,self.pos_argo)
        w,x,y,z = label['rotation']['w'],label['rotation']['x'],label['rotation']['y'],label['rotation']['z']
        self.q = np.array([x, y, z, w])       
        self.rot_mat_argo = Rotation.from_quat(self.q).as_dcm()
        
        
        self.ry = -Rotation.from_quat(self.q).as_euler('xyz')[-1] + np.pi/2.
        self.score = -1.0
        self.level_str = None
        self.level = self.get_obj_level()

    def get_obj_level(self):
        # Orginal : Assign level based on height of bounidng box in image, truncation, and occulusion value
        
        # Modified: Assign level based on distance from Origin of Lidar. Done
        distance = np.linalg.norm(self.pos)

        if distance <= 30.0:
            self.level_str = 'Easy'
            return 1  # Easy
        elif distance > 30.0 and distance <= 60.0:
            self.level_str = 'Moderate'
            return 2  # Moderate
        elif distance > 60 :
            self.level_str = 'Hard'
            return 3  # Hard
        else:
            self.level_str = 'UnKnown'
            return 4
        
    def generate_corners3d(self):
        """
        generate corners3d representation for this object
        :return corners_3d: (8, 3) corners of box3d in camera coord
        """
        l, h, w = self.l, self.h, self.w
        x_corners = [l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2]
        y_corners = [h/2., h/2., h/2., h/2., -h/2., -h/2., -h/2., -h/2.]
        z_corners = [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2]

        R = np.array([[np.cos(self.ry), 0, np.sin(self.ry)],
                      [0, 1, 0],
                      [-np.sin(self.ry), 0, np.cos(self.ry)]])
        corners3d = np.vstack([x_corners, y_corners, z_corners])  # (3, 8)
        corners3d = np.dot(R, corners3d).T
        corners3d = corners3d + self.pos
        return corners3d

    def to_bev_box2d(self, oblique=True, voxel_size=0.1):
        """
        :param bev_shape: (2) for bev shape (h, w), => (y_max, x_max) in image
        :param voxel_size: float, 0.1m
        :param oblique:
        :return: box2d (4, 2)/ (4) in image coordinate
        """
        if oblique:
            corners3d = self.generate_corners3d()
            xz_corners = corners3d[0:4, [0, 2]]
            box2d = np.zeros((4, 2), dtype=np.int32)
            box2d[:, 0] = ((xz_corners[:, 0] - Object3d.MIN_XZ[0]) / voxel_size).astype(np.int32)
            box2d[:, 1] = Object3d.BEV_SHAPE[0] - 1 - ((xz_corners[:, 1] - Object3d.MIN_XZ[1]) / voxel_size).astype(np.int32)
            box2d[:, 0] = np.clip(box2d[:, 0], 0, Object3d.BEV_SHAPE[1])
            box2d[:, 1] = np.clip(box2d[:, 1], 0, Object3d.BEV_SHAPE[0])
        else:
            box2d = np.zeros(4, dtype=np.int32)
            # discrete_center = np.floor((self.pos / voxel_size)).astype(np.int32)
            cu = np.floor((self.pos[0] - Object3d.MIN_XZ[0]) / voxel_size).astype(np.int32)
            cv = Object3d.BEV_SHAPE[0] - 1 - ((self.pos[2] - Object3d.MIN_XZ[1]) / voxel_size).astype(np.int32)
            half_l, half_w = int(self.l / voxel_size / 2), int(self.w / voxel_size / 2)
            box2d[0], box2d[1] = cu - half_l, cv - half_w
            box2d[2], box2d[3] = cu + half_l, cv + half_w

        return box2d

    def to_str(self):
        print_str = '%s %.3f %.3f %.3f box2d: %s hwl: [%.3f %.3f %.3f] pos: %s ry: %.3f' \
                     % (self.cls_type, self.trucation, self.occlusion, self.alpha, self.box2d, self.h, self.w, self.l,
                        self.pos, self.ry)
        return print_str

    def to_kitti_format(self):
        kitti_str = '%s %.2f %d %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f' \
                    % (self.cls_type, self.trucation, int(self.occlusion), self.alpha, self.box2d[0], self.box2d[1],
                       self.box2d[2], self.box2d[3], self.h, self.w, self.l, self.pos[0], self.pos[1], self.pos[2],
                       self.ry)
        return kitti_str


def get_objects_from_kitti_label(label_file):
    with open(label_file, 'r') as f:
        lines = f.readlines()
    objects = [Object3d_kitti(line) for line in lines]
    return objects

def get_objects_from_label(label_file):
    
    f = open(label_file)
    label_data = json.load(f) 
    objects = [Object3d(data) for data in label_data]
    return objects


In [41]:
rcnn_label_path = '/data/Argoverse/argoverse-tracking/val'

In [42]:
data_loader = ArgoverseTrackingLoader(os.path.join(rcnn_label_path))

In [43]:
gt_label = data_loader.label_list

In [44]:
gtobj_list = [get_objects_from_label(obj) for obj in gt_label]

In [45]:
gt_boxes3d = np.array([[obj.pos[0],obj.pos[1],obj.pos[2],obj.h,obj.w,obj.l,obj.ry] for obj in gtobj_list[pc_idx]])

In [46]:
gt_corners = kitti_utils.boxes3d_to_corners3d(gt_boxes3d, rotate=True)

In [47]:
predicted_labels = get_objects_from_kitti_label(os.path.join(rcnn_path,str(pc_idx).zfill(6)+'.txt'))

In [48]:
p_boxes3d = np.array([[obj.pos[0],obj.pos[1],obj.pos[2],obj.h,obj.w,obj.l,obj.ry] for obj in predicted_labels])

In [49]:
p_corners = kitti_utils.boxes3d_to_corners3d(p_boxes3d, rotate=True)

In [50]:
bounding_box = []

for corners in gt_corners:
    C1,C2,C3,C4,C5,C6,C7,C8 = corners.tolist()
    bounding_box = createBBox(bounding_box,C1,C2,C3,C4,C5,C6,C7,C8,color = 'blue')

for corners in p_corners:
    C1,C2,C3,C4,C5,C6,C7,C8 = corners.tolist()
    bounding_box = createBBox(bounding_box,C1,C2,C3,C4,C5,C6,C7,C8,color = 'green')

In [51]:
seg_pc = get_seg_lidar(pc_idx)
predicted_seg_pc = seg_pc[:,:3][seg_pc[:,4] == 1]

visualize(orig_pc, bounding_box)

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-0.11703711748123169, 0.5595594644546509, 91…